In [31]:
# import libraries
import math
import scipy
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.integrate as integrate

%matplotlib inline
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [76]:
# constants in SI
m = 9.10938291E-31 # mass of electron
q = 1.60217657E-19 # charge of electron
eV = 1.60217657E-19 # 1 electron volt in joules
ε = 8.85418782E-12 # permittivity of free space
h = 6.62606957E-34 # planck number
hbar = 1.054571800E-34 # reduced planck constant
Rydberg = 2.1787E-18 # Rydberg energy (in joules)
avogadro = 6.02214179E23
c = 299792458 # m/s

# units in SI
Å = 1E-10 # m to angstrom
k = 1.38064852E-23

In [80]:
E = np.linspace(0.5*eV,5*eV,501)

In [170]:
#materials parameters
E_g_ev = 1.52 #WSe2 (need to confirm) eV
E_g = E_g_ev*eV
m_c = 0.1*m #conduction band effective mass
m_v = 0.1*m #valence band effective mass



Start by modelling light incident on solar cell 

(For high concentrations, take it is a black body at 5760 Kelvin), the incoming particle flux is given by:

$F_{a} = \int_{E_{g}}^{\infty} \frac{2\Omega}{h^{3}c^{2}}\frac{E^{2}dE}{e^{ E/kT_{s}}-1}$

In [171]:
Sigma = 6.8e-3 #solid angle of light, 100 suns, dependent upon concentration
T_s = 5760 #K


In [172]:
kk = np.exp(E/(k*T_s))-1
conc= 2*Sigma/(h**3*c**2)
bb_rad = conc*(E**2)/kk

In [173]:
F_a_bb = np.trapz(E[E>E_g],bb_rad[E>E_g])

In [174]:
fig = plt.figure()
ax=fig.add_subplot(111, label="1")

ax.plot(E/eV, bb_rad*eV, 'b')# needs some constants in front as this includes the solid angle incident on the solar cell

ax.set_xlabel('E')
ax.set_ylabel('W$\mathregular{m^{-2}eV^{-1}}$')
ax.set_title('blackbody radiation')

plt.show()

In [ ]:
#Or - use AM 1.5 for 1 sun concentration (ASTMG173 global spectrum from 'American Society for Testing and Materials (ASTM) Terrestrial Reference Spectra for Photovoltaic Performance Evaluation')

wavelength, spectral_irr_w = np.genfromtxt('ASTMG173.csv', delimiter=',').T #spectral irradience (W/m^2/nm-1)
E_data = h*c/(wavelength*1E-9) # convert wavelength to energy including jacobian transform

spectral_irr = spectral_irr_w*1E9*h*c/((E_data)**2) #W/m^2J -include jacobian transform

spec_interp = interp1d(E_data[::-1], spectral_irr)
spec = spec_interp(E[E>=E_g])


In [ ]:
#plot Am1.5 in energy and wavelength

fig=plt.figure()
ax=fig.add_subplot(111, label="1")
ax2=fig.add_subplot(111, label="2", frame_on=False)

ax.plot(wavelength, spectral_irr_w, color='b')
ax.set_xlabel('Wavelength (nm)', color='b')
ax.set_ylabel('W$\mathregular{m^{-2}nm^{-1}}$',color='b')
ax.tick_params('x', colors='b')
ax.tick_params('y', colors='b')

ax2.plot(E_data/eV, spectral_irr*eV, color='r')
ax2.xaxis.tick_top()
ax2.yaxis.tick_right()
ax2.set_ylabel('W$\mathregular{m^{-2}eV^{-1}}$',  color='r')
ax2.set_xlabel('Energy (eV)', color='r')
ax2.tick_params('x', colors='r')
ax2.tick_params('y', colors='r')
ax2.xaxis.set_label_position('top') 
ax2.yaxis.set_label_position('right') 

fig.tight_layout()
plt.show()


In [ ]:
F_a_ASTMG = np.trapz(spectral_irr[E_data>E_g],E_data[E_data>E_g]) #Need to add solid angle here - currently this is wrong number i think?

In [ ]:
#carrier temperature (insert rate equation here)
T_c = 500 #K

In [ ]:
#calculate number of electrons in conduction band - based on F_a
N = 1E24
#holes in valence band
P = 1E24

In [ ]:
#calculate quasi fermi levels from Nilsson approximation, cite Bennett, Soref, Alamo 1990

In [ ]:
N_c = 2*(2*np.pi*m_c*k*T_c/(h**2))**(3/2)
N_v = 2*(2*np.pi*m_v*k*T_c/(h**2))**(3/2) #effective densities of states in conduction and valence band

In [ ]:
#Nilsson approx
Mu_c = k*T_c*(np.log(N/N_c)+(N/N_c)*(64+0.05524*(N/N_c)*(64+(N/N_c)**(1/2))**(-1/4)))
Mu_v = k*T_c*(-1*(np.log(P/N_v)+(P/N_v)*(64+0.05524*(P/N_v)*(64+(P/N_v)**(1/2))**(-1/4)))-E_g)

In [ ]:
Delta_mu = Mu_c-Mu_v
print(Mu_c/eV, Mu_v/eV, Delta_mu/eV)


outgoing radiation is given by
$F_{b} = \int_{E_{g}}^{\infty} \frac{2\pi}{h^{3}c^{2}}\frac{E^{2}dE}{e^{(E-\Delta\mu)/kT_{C}}-1}$

In [ ]:
F_b = np.trapz(E[E>E_g],2*pi/(h**3c**2)*(E[E>E_g]**2/(np.exp((E[E>E_g]-Delta_mu)/(k*T_c))-1))